In [35]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [36]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('DiabetesBinaryClassification.csv')
# df = pd.read_csv('data.csv', header=None)

# Preprocessing data
df = df.drop_duplicates()
df = df.dropna()
for series_name, series in df.items():
  df[series_name] = series.astype('float')


X = []
y = []
for data in df.values:
    X.append( np.array( data[ : -1 ], dtype = float ) )
    y.append( [ data[ -1 ] ] )

    from sklearn.model_selection import train_test_split

x = np.asarray( X )
y = np.asarray( y )

for idn,n in np.ndenumerate(y):
  if n == 0:
    y[idn] = -1

# X = np.array(df.iloc[:, [0,1]])
# y = np.array(df.iloc[:, 2])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [37]:
from cvxopt import matrix, solvers

def fit_model(X_train: np.array, y_train: np.array, C: float=0.1) -> tuple[int,int]:
  X_train = np.asarray( X_train )
  m, n = X_train.shape
  K = np.matmul(X_train, X_train.T)
  P = matrix(np.matmul(y_train,y_train.T) * K)
  q = matrix(np.ones((m, 1)) * -1)
  A = matrix((y_train.reshape(1, -1)))
  b = matrix(np.zeros(1))          
  G = matrix(np.vstack((np.eye(m) * -1, np.eye(m))))        
  h = matrix(np.hstack((np.zeros(m), np.ones(m) * C)))

  sol = solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol["x"])

  w = np.dot((y_train * alphas).T, X_train)[0]
  S = (alphas > 1e-5).flatten()
  b = np.mean(y_train[S] - np.dot(X_train[S], w.reshape(-1,1)))
  return w,b

w,b = fit_model(X_train, y_train)

     pcost       dcost       gap    pres   dres
 0: -1.3704e+02 -9.7050e+01  4e+03  3e+01  2e-10
 1: -2.2432e+01 -9.2442e+01  2e+02  1e+00  2e-10
 2: -1.6742e+01 -5.4403e+01  5e+01  2e-01  3e-11
 3: -1.7610e+01 -2.4022e+01  7e+00  2e-02  2e-11
 4: -1.9137e+01 -2.1206e+01  2e+00  5e-03  2e-11
 5: -1.9605e+01 -2.0550e+01  1e+00  1e-03  2e-11
 6: -1.9804e+01 -2.0266e+01  5e-01  6e-04  2e-11
 7: -1.9947e+01 -2.0068e+01  1e-01  1e-04  3e-11
 8: -1.9991e+01 -2.0010e+01  2e-02  1e-05  3e-11
 9: -1.9999e+01 -2.0001e+01  2e-03  1e-06  3e-11
10: -2.0000e+01 -2.0000e+01  4e-05  3e-08  2e-11
11: -2.0000e+01 -2.0000e+01  8e-07  3e-10  3e-11
Optimal solution found.


In [38]:
from sklearn import metrics

def evaluate_model(X_test: np.array, y_test: np.array, w: np.array, b: float):
  X_test = np.asarray( X_test )
  n_test = X_test.shape[0]
  prod = np.dot(X_test,w) + np.full(n_test,b)
  y_pred = np.sign(prod)

  # Model Accuracy: how often is the classifier correct?
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

evaluate_model(X_test, y_test, w, b)

Accuracy: 0.7545691906005222


In [39]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

/home/manojturaga/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.762402088772846
Precision: 0.7263157894736842
Recall: 0.5149253731343284
